In [ ]:
#installing kaggle library
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"khushi2508","key":"7bd12c4e35445d4e147af58a81bce8aa"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d jp797498e/twitter-entity-sentiment-analysis

Dataset URL: https://www.kaggle.com/datasets/jp797498e/twitter-entity-sentiment-analysis
License(s): CC0-1.0
  0% 0.00/1.99M [00:00<?, ?B/s]
100% 1.99M/1.99M [00:00<00:00, 114MB/s]


In [ ]:
!unzip twitter-entity-sentiment-analysis.zip -d /content/

Archive:  twitter-entity-sentiment-analysis.zip
  inflating: /content/twitter_training.csv  
  inflating: /content/twitter_validation.csv  


In [ ]:
# Imports
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization, Embedding, Dense, Dropout, GlobalAveragePooling1D
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#Loading the CSV file from pandas dataframe
column_names = ['ID','Information','Target','Text']
train_data = pd.read_csv('/content/twitter_training.csv',header=None,names=column_names)
test_data = pd.read_csv('/content/twitter_validation.csv',header=None,names=column_names)

In [ ]:
train_data.head()

,ID,Information,Target,Text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [ ]:
test_data.head()

,ID,Information,Target,Text
0,3364,Facebook,Irrelevant,I mentioned on Facebook that I was struggling ...
1,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
2,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
3,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
4,4433,Google,Neutral,Now the President is slapping Americans in the...


In [ ]:
print("Train data shape : ",train_data.shape)
print("Test Data Shape : ",test_data.shape)
print("Train data size : ",train_data.size)
print("Test Data size : ",test_data.size)

Train data shape :  (74682, 4)
Test Data Shape :  (1000, 4)
Train data size :  298728
Test Data size :  4000


In [ ]:
X_train = train_data["Text"]
y_train = train_data["Target"]
X_test = test_data["Text"]
y_test = test_data["Target"]

In [ ]:
#Creating Preprocessing function for text
stop_words = set(stopwords.words('english'))
def preprocess_text(text):
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)  # Remove URLs
    text = re.sub(r'\@\w+|\#','', text)  # Remove mentions and hashtags
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = text.lower()  # Lowercase
    text = " ".join([word for word in text.split() if word not in stop_words])  # Remove stopwords
    return text

In [ ]:
import re
import numpy as np
from nltk.corpus import stopwords
# Download stopwords if not already present
import nltk
nltk.download('stopwords')

#Creating Preprocessing function for text
stop_words = set(stopwords.words('english'))
def preprocess_text(text):
    # Convert input to string to ensure compatibility with re.sub
    text = str(text)
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)  # Remove URLs
    text = re.sub(r'\@\w+|\#','', text)  # Remove mentions and hashtags
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = text.lower()  # Lowercase
    text = " ".join([word for word in text.split() if word not in stop_words])  # Remove stopwords
    return text



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#Preprocessing the test and train text
# Convert the generator expression to a list comprehension to evaluate it immediately.
X_train = np.array([preprocess_text(text) for text in X_train])
X_test = np.array([preprocess_text(text) for text in X_test])

In [ ]:
# Encode Labels (Ternary Classification)
le = LabelEncoder()
y_train = le.fit_transform(y_train)  # Convert labels to numeric format
label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))  # Save mapping for reference
print("Label Mapping:", label_mapping)  # {'negative': 0, 'neutral': 1, 'positive': 2, 'irrelevant': 3}

y_test = le.transform(y_test)  # Ensure labels are encoded similarly

Label Mapping: {'Irrelevant': 0, 'Negative': 1, 'Neutral': 2, 'Positive': 3}


In [ ]:
# Define the TextVectorization layer for tokenizing and vectorizing
max_vocab_size = 10000
max_sequence_length = 100

vectorizer = TextVectorization(max_tokens=max_vocab_size, output_mode='int', output_sequence_length=max_sequence_length)
vectorizer.adapt(X_train)  # Adapt vectorizer to training data


In [ ]:
# Build the Model
model = Sequential([
    vectorizer,
    Embedding(input_dim=max_vocab_size, output_dim=128),
    GlobalAveragePooling1D(),
    Dropout(0.2),
    Dense(64, activation="relu"),
    Dropout(0.2),
    Dense(4, activation="softmax")  # For ternary classification; change to 'sigmoid' if binary-classification
])

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Step 4: Train the Model
import numpy as np

# Assuming X_train is a Pandas Series or DataFrame
# Convert to NumPy array if it's not already
X_train = np.array(X_train, dtype=object) # Explicitly convert to string dtype

# Step 4: Train the Model
history = model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
2334/2334 ━━━━━━━━━━━━━━━━━━━━ 40s 15ms/step - accuracy: 0.3394 - loss: 1.3348
Epoch 2/10
2334/2334 ━━━━━━━━━━━━━━━━━━━━ 42s 15ms/step - accuracy: 0.5797 - loss: 1.0187
Epoch 3/10
2334/2334 ━━━━━━━━━━━━━━━━━━━━ 40s 15ms/step - accuracy: 0.6585 - loss: 0.8620
Epoch 4/10
2334/2334 ━━━━━━━━━━━━━━━━━━━━ 37s 16ms/step - accuracy: 0.7073 - loss: 0.7501
Epoch 5/10
2334/2334 ━━━━━━━━━━━━━━━━━━━━ 41s 16ms/step - accuracy: 0.7282 - loss: 0.6998
Epoch 6/10
2334/2334 ━━━━━━━━━━━━━━━━━━━━ 38s 14ms/step - accuracy: 0.7369 - loss: 0.6686
Epoch 7/10
2334/2334 ━━━━━━━━━━━━━━━━━━━━ 43s 15ms/step - accuracy: 0.7464 - loss: 0.6472
Epoch 8/10
2334/2334 ━━━━━━━━━━━━━━━━━━━━ 37s 16ms/step - accuracy: 0.7646 - loss: 0.6077
Epoch 9/10
2334/2334 ━━━━━━━━━━━━━━━━━━━━ 36s 16ms/step - accuracy: 0.7647 - loss: 0.5990
Epoch 10/10
2334/2334 ━━━━━━━━━━━━━━━━━━━━ 43s 16ms/step - accuracy: 0.7811 - loss: 0.5687


In [ ]:
# Step 5: Model Evaluation
# Predict on test data
X_test = np.array(X_test, dtype=object) # Explicitly convert to string dtype
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


In [ ]:
# Generate a classification report
print(classification_report(y_test, y_pred_labels, target_names=le.classes_))

              precision    recall  f1-score   support

  Irrelevant       0.88      0.80      0.84       172
    Negative       0.79      0.93      0.86       266
     Neutral       0.97      0.68      0.80       285
    Positive       0.78      0.94      0.85       277

    accuracy                           0.84      1000
   macro avg       0.86      0.84      0.84      1000
weighted avg       0.86      0.84      0.84      1000



In [ ]:
model.save("sentiment_model.h5")

In [ ]:
model.save('sentiment_model.keras')

In [ ]:
import tensorflow as tf

# Load the Keras model
sentiment_model = tf.keras.models.load_model('sentiment_model.keras')

# Convert the Keras model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(sentiment_model)
# Enable TF Select to support unsupported operations
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
tflite_model = converter.convert()

# Save the TensorFlow Lite model
with open("sentiment_model.tflite", "wb") as f:
    f.write(tflite_model)


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 7 variables whereas the saved optimizer has 12 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Saved artifact at '/tmp/tmpoaidonz5'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None,), dtype=tf.string, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  132042553604960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132041730513680: TensorSpec(shape=(), dtype=tf.int64, name=None)
  132042374144144: TensorSpec(shape=(), dtype=tf.string, name=None)
  132042374147488: TensorSpec(shape=(), dtype=tf.int64, name=None)
  132042357752224: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132041733894592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132041733882096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132041733888256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132042328332720: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [ ]:
# Extract vocabulary and save
import json
vocabulary = vectorizer.get_vocabulary()
with open("vocabulary.json", "w") as f:
    json.dump(vocabulary, f)


In [ ]:
with open("label_mapping.json", "w") as f:
    # Convert NumPy int64 to Python int before dumping to JSON
    modified_label_mapping = {k: int(v) for k, v in label_mapping.items()}
    json.dump(modified_label_mapping, f)  # Save label to index mappings